In [1]:
import pandas as pd
import data_util
from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from state_util import StateUtil
from tec_an import TecAn
from data_agent import *
import numpy as np
from data_util import *
import tensorflow as tf
from sklearn_model_hyper import *
import pandas as pd
from stock_agent import *
from backtest import *
from joblib import dump, load
from datetime import datetime, timedelta
from tec_an import *
from bitstamp import *
from process_live_trade import *
from model_winner_select import *
from eval_model import *

base_dir = "data/"

In [2]:
currency = "btcusd"

full_data = base_dir + "backtest/"
full_data = base_dir + "4jul21/"
#full_data = base_dir + "btcusd/"
data_gen = DataGenerator(random = False, base_dir = full_data)
data_gen.rewind()
data_count = (data_gen.steps - 100)
#data_count = 200000
raw = data_gen.next()

minute = 5

hot_load = True

In [3]:
path = "model/"
files = os.listdir(path)
models = []
for file in files:
    models.append(f"{path}{file}")

models = ['model/rand']
#models = ['model/quadra']
    
timestamp = int(raw['timestamp'])

In [4]:
result = load(models[-1])[-1]

In [5]:
model = result['model']
window = result['window']
minutes = result['minutes']
step = result['step']
profit = result['profit']

In [6]:
val_start = 1629215891
val_end = 1629285606

In [7]:

online = load_val_data(minutes = 5, 
                 window = window, 
                 val_start = val_start,
                 val_end = val_end,
                 currency_list = [currency])


https://www.bitstamp.net/api/v2/ohlc/btcusd/?&currency_pair=btcusd&step=300&limit=1000&start=1629215891


  0%|          | 0/235 [00:00<?, ?it/s]

In [8]:

eval_model(model, currency, step, online, hot_load_total = 300)


BackTest (pending_sell_steps=1 sell_on_profit=True value=100)
https://www.bitstamp.net/api/v2/ohlc/btcusd/?&currency_pair=btcusd&step=300&limit=1000&start=1629195900


  0%|          | 0/301 [00:00<?, ?it/s]

-0.148% -> 99.85200100840729
Positive: 17 Negative: 8


(<stock_agent.BackTest at 0x7fc17ff7d9a0>,
 {'recall': 0.1323529411764706,
  'precision': 0.23076923076923078,
  'f1': 0.16822429906542055,
  'accuracy': 0.6212765957446809,
  'roc_auc': 0.47635611130679817})

In [9]:

eval_model(model, currency, step, online, hot_load_total = 500)


BackTest (pending_sell_steps=1 sell_on_profit=True value=100)
https://www.bitstamp.net/api/v2/ohlc/btcusd/?&currency_pair=btcusd&step=300&limit=1000&start=1629135900


  0%|          | 0/501 [00:00<?, ?it/s]

0.20057% -> 100.20057010706546
Positive: 26 Negative: 12


(<stock_agent.BackTest at 0x7fc17ff8a8b0>,
 {'recall': 0.14705882352941177,
  'precision': 0.25,
  'f1': 0.18518518518518517,
  'accuracy': 0.625531914893617,
  'roc_auc': 0.4837090524832688})

In [ ]:

eval_model(model, currency, step, online, hot_load_total = 1000)


BackTest (pending_sell_steps=1 sell_on_profit=True value=100)
https://www.bitstamp.net/api/v2/ohlc/btcusd/?&currency_pair=btcusd&step=300&limit=1000&start=1628985900
https://www.bitstamp.net/api/v2/ohlc/btcusd/?&currency_pair=btcusd&step=300&limit=1000&start=1629285600


  0%|          | 0/1001 [00:00<?, ?it/s]

In [ ]:

eval_model(model, currency, step, online, hot_load_total = 3000)


In [ ]:
winner = get_best_model(
    minutes_list=[minute],
    result_paths=models,
    currency_list=[currency],
    timestamp = timestamp,
    winner_path = None
)

In [ ]:
result = winner
if (winner == None):
    result = results[-1]
model = result['model']
window = result['window']
minutes = result['minutes']
step = result['step']
profit = result['profit']
print(f"Minutes={minutes} Window={window} Step={step} | {profit}")
print(f"{model}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
def get_charts(stock_data):
    fig = plt.figure()
    ax = plt.axes()
    x = stock_data
    plt.plot(x, linewidth=1)
    #plt.grid(b=True, color='aqua', alpha=0.3, linestyle='-.', linewidth=2)
    plt.show()


In [ ]:

on_raw_data = lambda raw: print(raw)

data_gen.rewind()

first = data_gen.next()

In [ ]:
agent_reference, back_reference, stock_reference = get_agent(minutes = minutes,
                                win = window,
                                step = step,
                                currency = currency,
                                hot_load = hot_load,
                                model = model,
                                timestamp = int(first['timestamp']),
                                simulate_on_price = True)
prices = []
agent_reference.on_closed_price = lambda price: prices.append(price)
#agent.ohlc['open'].plot()

In [ ]:
data_gen.rewind()


reference_list = []
#data_count
for i in tqdm(range(data_count)):
    raw = data_gen.next()
    price = raw[PRICE_KEY]
    amount = raw[AMOUNT_KEY]
    timestamp = raw[TIMESTAMP_KEY]
    timestamp_pd = pd.to_datetime(timestamp, unit='s')
    reference_list.append([timestamp_pd, price])


In [ ]:
DATE = 'Date'
df = pd.DataFrame(agent_reference.list, columns = [DATE, CLOSE])
df = df.set_index(pd.DatetimeIndex(df[DATE]))

In [ ]:
time = df[CLOSE].resample(agent_reference.resample)
ohlc = time.ohlc()
print("All data reference")
#ohlc.plot()
ohlc

In [ ]:
reference_list[0]

In [ ]:
agent_raw, back_raw, stock_raw = get_agent(minutes = minutes,
                                win = window,
                                step = step,
                                currency = currency,
                                hot_load = hot_load,
                                model = model,
                                timestamp = int(first['timestamp']),
                                simulate_on_price = True)
prices = []

In [ ]:
data_gen.rewind()
agent_raw.save_history = True
#data_count
for i in range(data_count):
    raw = data_gen.next()
    agent_raw.on_new_raw_data(raw)

In [ ]:
val_start = int(first['timestamp'])
val_end = int(raw['timestamp'])

start_string = pd.to_datetime(val_start, unit='s')
end_string = pd.to_datetime(val_end, unit='s')

print(f"{start_string}({val_start}) -> {end_string}({val_end})")

In [ ]:
back_raw.on_down(back_raw.buy_price, back_raw.buy_price)
back_raw.report()

In [ ]:
agent_raw.ohlc[-50:]["open"].plot()
#agent.ohlc[1:]

In [ ]:
tec = TecAn(windows = window, windows_limit = 100)
source_data_generator = SourceDataGenerator(tec = tec)
online = get_online_data(minutes = 5, 
                 source_data_generator = source_data_generator, 
                 load_from_disk = True, 
                 file_prefix = "____",
                 val_start = val_start,
                 val_end = val_end)


In [ ]:
online.load_val_cache([currency],1629215891,  1629229205)

In [ ]:
x_list, price_list, time_list = online.load_val_data(currency)

In [ ]:
get_charts(price_list)
print(len(price_list))

In [ ]:
agent_array, back_array, stock_array = get_agent(minutes = minutes,
                                win = window,
                                step = step,
                                currency = currency,
                                hot_load = hot_load,
                                model = model,
                                timestamp = int(first['timestamp']),
                                simulate_on_price = True)

In [ ]:
agent_array.save_history = True
for idx in range(len(price_list)):
    price = price_list[idx]
    time = time_list[idx]
    order = [[f"{price}", f"{price}"]]
    amount = 0.0
    agent_array.process_data(price, amount, time, order, order)

In [ ]:
back_array.on_down(back_array.buy_price, back_array.buy_price)
back_array.report()

In [ ]:
def eval_step_(model, train_set, step, provider, verbose = False):

    valX, valY, times = provider.load_val_data(train_set)
    
    x, y, closed_prices = get_sequencial_data(valX, valY, step)
    
    history = []
    
    preds = model.predict(np.array(x))
    
    #for idx in range(len(x)):
    #    print(f"{times[idx]} -> {closed_prices[idx]} {y[idx]} ")

    metrics = {}
    metrics["recall"] = recall_score(y, preds)
    metrics["precision"] = precision_score(y, preds)
    metrics["f1"] = f1_score(y, preds)
    metrics["accuracy"] = accuracy_score(y, preds)
    metrics["roc_auc"] = roc_auc_score(y, preds)
    
    back = BackTest(value = 100, 
                    verbose = True, 
                    sell_on_profit = True,
                    pending_sell_steps = step)
    
    back = backtest_model(model, valX, valY, back)
    
    for idx in range(len(times)):
        timestamp = times[idx]
        price = closed_prices[idx]
        y = model.predict([valX[idx]])
        is_up = False
        if (y[0] > 0.5):
            is_up = True
        history.append((timestamp, price, x, is_up))
    
    get_charts(valY)

    
    return back, metrics, history
    

In [ ]:
back_eval, metr, history = eval_step_(model, currency, step, online, verbose = False)
back_eval.on_down(back_eval.buy_price, back_eval.buy_price)
back_eval.report()

In [ ]:
for data in history:
    print(f"{data[0]} - {data[1]} - {data[3]}")

In [ ]:
print(metrics)


In [ ]:
back_raw.report()
#-1.78465% -> 98.21535030676218

In [ ]:
back_array.report()
#-1.89717% -> 98.10282664807247

In [ ]:
agent_raw.report()

In [ ]:
agent_array.report()